In [49]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bertviz import head_view

# Path to your fine-tuned model
model_path = '/datasets/mdawood/swahili-xlmr-binary-classification-base'

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, output_attentions=True)

# Sample Swahili text (replace with your own example)
text = "Leo rais alitoa hotuba muhimu kuhusu maendeleo ya nchi"

# Tokenize the input
inputs = tokenizer(text, return_tensors='pt')

# Get model outputs
outputs = model(**inputs)

# Extract attention
attention = outputs.attentions

# Get tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Generate and save the head view visualization
html_head_view = head_view(attention, tokens, html_action='return')

with open('head_view.html', 'w', encoding='utf-8') as f:
    f.write(html_head_view.data)

print("Head View saved to head_view.html")

# Display the visualization in the notebook
from IPython.display import display
display(html_head_view)

Head View saved to head_view.html


In [54]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.core.display import display, HTML
import re
# Load the model and tokenizer
model_path = '/datasets/mdawood/swahili-xlmr-binary-classification-base'  # Adjust the path if necessary
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, output_attentions=True)
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to classify the sentence (True/False)
def classify_sentence(text, model, tokenizer):

    # Clean the text
    text = clean_text(text)

    # Tokenize the input text
    inputs = tokenizer.encode_plus(
        text,
        return_tensors='pt',
        add_special_tokens=True,
        return_attention_mask=True
    )

    # Move inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Run the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract logits and make prediction
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # 0 = False, 1 = True (Assumed binary classification setup)
    return "True" if predicted_class == 1 else "False"

def clean_text(text):
    # Use regex to remove everything except letters and whitespace
    cleaned_text = re.sub(r'[^A-Za-z\s]', '', text)
    return cleaned_text

# Function to highlight important words in a sentence
def highlight_text(text, model, tokenizer):

    # Clean the text
    text = clean_text(text)

    # Tokenize the input text
    inputs = tokenizer.encode_plus(
        text,
        return_tensors='pt',
        add_special_tokens=True,
        return_attention_mask=True
    )

    # Move inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Run the model and get attentions
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract attentions
    attentions = outputs.attentions  # Tuple of (num_layers,) each of shape (batch_size, num_heads, seq_length, seq_length)

    # Aggregate attentions
    num_layers = len(attentions)
    seq_length = attentions[0].shape[2]

    attention_scores = torch.zeros(seq_length, seq_length).to(device)

    for layer_attention in attentions:
        # Sum over heads
        head_sum = layer_attention[0].sum(dim=0)
        attention_scores += head_sum

    # Average over layers
    attention_scores = attention_scores / num_layers

    # Compute token importance scores
    token_importance = attention_scores.sum(dim=0)
    token_importance = token_importance / token_importance.sum()

    # Convert input IDs to tokens
    input_ids = inputs['input_ids'][0]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Merge subword tokens and their scores
    merged_tokens = []
    merged_scores = []
    current_token = ''
    current_score = 0.0
    token_count = 0

    for token, score in zip(tokens, token_importance):
        token = token.replace('</s>', '').replace('<s>', '')
        if token == '':
            continue
        if token.startswith('▁') or token.startswith('##'):
            # New word
            if current_token != '':
                # Append the previous word
                merged_tokens.append(current_token)
                merged_scores.append(current_score / token_count)
            # Start a new word
            current_token = token.lstrip('▁').lstrip('##')
            current_score = score.item()
            token_count = 1
        else:
            # Continuation of the current word
            current_token += token
            current_score += score.item()
            token_count += 1

    # Append the last word
    if current_token != '':
        merged_tokens.append(current_token)
        merged_scores.append(current_score / token_count)

    # Map scores to colors
    norm = colors.Normalize(vmin=min(merged_scores), vmax=max(merged_scores))
    cmap = cm.get_cmap('Reds')

    html_tokens = []
    for token, score in zip(merged_tokens, merged_scores):
        color = colors.rgb2hex(cmap(norm(score)))
        html_token = f"<span style='background-color:{color}'>{token}</span>"
        html_tokens.append(html_token)

    html_text = ' '.join(html_tokens)

    return html_text

swahili_sentences = [
    "BUNGE limehakikishiwa kuwa hakuna changamoto inayoweza kufarakanisha, kutenganisha wala kurudisha nyuma Muungano wa Tanganyika na Zanzibar; na kwamba badala yake unaendelea kuimarika na kushamiri.Aidha, Waziri wa Nchi, Ofisi ya Makamu wa Rais- Muungano na Mazingira, January Makamba ameasa dhidi ya wanaotumia changamoto kuupa jina baya au kuhoji uhalali wa Muungano. January alitoa msimamo huo jana wakati akiwasilisha bungeni Makadirio ya Mapato na Matumizi kwa Mwaka 2019/20 na kusema changamoto zilikuwapo, zipo na zitaendelea kuwapo isipokuwa jambo la muhimu ni kuwa na utaratibu bora wa kuzishughulikia.Alisema chini ya uongozi wa Rais John Magufuli, Makamu wa Rais, Samia Suluhu Hassan na Rais wa Zanzibar, Dk Ali Mohamed Shein, ofisi na serikali kwa ujumla itaendelea kufanyia kazi mambo yote yanayoleta changamoto katika utekelezaji wa masuala yanayohusu muungano. “Chini ya uongozi wao hakuna changamoto yoyote inayoweza kutufarakanisha, kututenganisha wala kuturudisha nyuma,” alisema.Aliongeza, “mwaka huu Muungano wetu unatimiza miaka 55 tangu kuasisiwa kwake. Natoa mwito kwa Watanzania wote kuendelea kuuthamini, kuuenzi, kulinda na kuudumisha ili jitihada za kusukuma mbele maendeleo kiuchumi, kisiasa na kijamii kwa faida za pande zote mbili.” Wakichangia hotuba ya Waziri Makamba, Mbunge wa Viti Maalumu, Josephine Genzabuke (CCM) alishauri elimu kuhusu umuhimu wa Muungano itolewe kuanzia darasa la kwanza hadi chuo kikuu. “Elimu kuhusu Muungano ianzie hapa bungeni,” alisema Mbunge wa Welezo, Dk Saada Mkuya (CCM).Mkuya alisema baadhi ya kauli zitolewazo bungeni zinatokana na watu kutofahamu maana ya Muungano. Alisisitiza kila mtu wakiwamo mawaziri, asimamie Muungano katika kazi zake za kila siku bila kujali kama wizara ni ya Muungano au la. Katika mwaka wa fedha 2019/20 ofisi binafsi ya Makamu wa Rais iliomba na kuidhinishiwa Sh bilioni 7.8 kwa ajili ya matumizi ya kawaida na matumizi mengineyo.Pia Ofisi ya Makamu wa Rais iliidhinishiwa Sh bilioni 5.3 kwa ajili ya matumizi ya mishahara na matumizi mengineyo.",
    "Mwenyekiti wa Kamati ya Maandalizi ya tamasha hilo, Alex Msama alisema jana kuwa amezungumza na waimbaji hao na wamekubaliana watatunga wimbo maalum watakaouimba kwa pamoja siku hiyo. Msama alisema wimbo huo utaitwa Amani na Upendo na kwamba maandalizi yanaendelea kuhakikisha wasanii wengi maarufu wanakuwepo.“Watakuwa wasanii maarufu, naamini wengi waliokuwepo kwenye Tamasha la Pasaka mwaka huu watashiriki katika wimbo huo. “Tunaendelea na mazungumzo na wasanii mbalimbali, ambapo tayari Rose Muhando ameshakubali kutumbuiza na wengine tuliowatangaza,” alisema Msama.Baadhi ya wasanii maarufu waliowahi kushiriki matamasha yanayoandaliwa na Msama ni Rose Muhando, Rebecca Malope, Christina Shusho, Upendo Nkone, Upendo Kilahiro, Solomon Mukubwa, Atosha Kissava, Ephraim Sekeleti na kundi la Glorious Celebration.Pia kwaya mahiri ya Kinondoni Revival, Mwinjilisti Faraja Ntaboba kutoka Jamhuri ya Kidemokrasia ya Congo (DRC) na Sohly Mahlangu.",
    "WAZ IR I Mkuu, Kassim Majaliwa ameondoka juzi jioni kwenda nchini China kwa ajili ya kuhudhuria mkutano wa Wakuu wa Nchi na Serikali wa Jukwaa la Ushirikiano kati ya China na Afrika (FOCAC).Mkutano huo wa siku mbili, unatarajiwa kuanza kesho Jumatatu jijini Beijing nchini China kwa kufunguliwa na Rais wa China, Xi Jinping.Katika mkutano huo, Waziri Mkuu anamwakilisha Rais John Magufuli.Kwa mujibu wa taarifa ya Ofisi ya Waziri Mkuu, mbali na kuhudhuria mkutano huo pia, Waziri Mkuu anatarajia kukutana na kufanya mazungumzo na viongozi wakuu wa kampuni mbalimbali akiwemo Rais wa Kampuni ya CCECC, Rais wa Shirika la Norinco, Rais wa Kampuni ya Zijin Gold Mine na Mwenyekiti wa Kampuni ya Saruji ya Hegnya.",
    "Na Raymond Minja Iringa Upande wa Jamhuri katika kesi ya Mwenyekiti wa Mtandao wa Wanafunzi Tanzania (TSNP), Abdul Nondo, Veronica Fredy anatarajiwa kuletwa makamani kutoa ushahidi katika kesi inayomkabili mwanafunzi huyo. Mbali na mpenzi huyo wa Nondo, Jamhuri pia inakusudia kuleta mashahidi wengine wanne ambao ni Koplo Salum, Alphonce Mwamule, Koplo Abdulkadir na mfanyakazi kutoka Kampuni ya Simu za Mkononi ya Tigo. Kesi hiyo ilikuja leo mbele ya hakimu Mkazi wa Wilaya ya Iringa, John Mpitanjia kwa ajili ya kusomewa maelezo ya awali ambapo Wakili wa Serikali, Abel Mwandalamo amesema Nondo anakabiliwa na mashtaka mawili ambayo ni kusambaza taarifa za uongo akiwa Ubungo jijini Dar Es Salaam na kutoa taarifa za uongo akiwa mjini Mafinga kwa askari Polisi kituo cha Mafinga Wilayani Mufindi. Wakili Mwandalamo amedai upande wa Jamhuri unakusudia kuambatanisha vielelezo vya ushahidi ambavyo ni taarifa za uchunguzi kuhusiana na mawasiliano ya simu, maelezo kutoka Mjini Mafinga yakidai Nondo katekwa na kuleta kielelezo cha simu za mkononi. Hakimu Mpita Njia ameahirisha kesi hiyo hadi Aprili 18, mwaka huu ambapo itaanza kusikilizwa kwa siku mbili mfululizo, kutokana na maombi ya upande wa utetezi ya Wakili Charles Luoga.",
    "CALIFORNIA, MAREKANI RAPA Shad Moss, maarufu kwa jina la Bow Wow, ametumia ukurasa wake wa Twitter na kudai kwamba, amekuwa mmoja kati ya wasanii ambao wanatengwa na kuchukiwa. Staa huyo ambaye alianza kuwa na jina kubwa tangu akiwa na umri mdogo, amesisitiza kwamba, maisha yake kwa sasa yamekuwa na changamoto nyingi. “Wala sijali, vyombo vya habari vimekuwa vikinisema vibaya, nimekuwa miongoni mwa watu wanaochukiwa sana, kila msichana ambaye nilimpa moyo wangu aliniangusha, baba yangu ni mlevi na sasa ni mgonjwa, mchumba wangu alipoteza mtoto kwa ajili ya mimba kuharibika, nimekuwa nikiguswa na kila kitu, akili yangu haipo sawa kabisa, naumia sana,” aliandika Bow Wow. Hata hivyo, kabla ya mashabiki wake kuanza kujibu ujumbe huo, aliamua kufuta Twitt hiyo, lakini baada ya muda aliendelea kuandika ujumbe mwingine kwa kumwelezea mpenzi wake, Kiyomi.",
    "MWANDISHI WETU-DODOMA      MAOFISA Lishe nchini wametakiwa kuelimisha wananchi kuhusu umuhimu wa kuzingatia lishe bora ili kuondoa changamoto ya utapiamlo na udumavu katika jamii. Akizungumza wakati akifunga mkutano wa siku mbili wa wadau wa lishe uliofanyika jijini Dodoma juzi, kwa niaba ya Waziri wa nchi, Ofisi Waziri mkuu, Kazi Vijana Ajira na Wenye Ulemavu, Jenista Mhagama, Katibu Mkuu Ofisi ya Waziri Mkuu anayeshughulikia Sera Uratibu na Uwekezaji,Dorothy Mwaluko alisema kuwa maofisa lishe wanapaswa kuongeza juhudi katika kutoa elimu kwa kushirikiana na wadau kuhusu umuhimu wa lishe bora. “Kipaumbele lazima kiwekwe katika makundi maalum kama wazee, watoto, akinamama wanajawazito na wale wanaonyonyesha ili kufanikisha azma ya Serikali kuondoa changamoto ya utapiamlo na udumavu hapa nchini,”alisema  Mwaluko Alisema kuwa Serikali inasisitiza matumizi ya sanaa, vyombo vya habari na viongozi wa kisiasa wakiwemo wabunge katika kuelimisha umma kuhusu athari za utapimlo na udumavu hapa nchini. Kwa upande wake Katibu mkuu, Wizara ya Afya, Maendeleo ya Jamii, Jinsia Wazee na Watoto (Afya), Dk. Zainab Chaula, alisema kuwa maazimio ya kikao hicho yanapaswa kutoa majawabu yatakayowezesha kuondokana na utapiamlo na udumavu hapa nchini. Alisema Serikali imeweka mazingira wezeshi ya kufanya kazi na wadau katika kukabiliana na changamoto ya utapiamlo na udumavu katika maeneo yote nchini ili kujenga jamii yenye afya bora. Dk. Chaula aliwataka washiriki wa mkutano huo kuhakikisha kuwa kila mmoja anakuwa sehemu ya mapambano dhidi ya utapiamlo na udumavu hapa nchini ili kujenga jamii yenye afya bora hasa wakati huu Taifa linapojenga uchumi wa viwanda na kutekeleza azma ya kuleta ustawi kwa wananchi.",
    "Halmashauri ya Wilaya ya Kisarawe inakaribisha wawekezaj wakajenge viwanda kwenye eneo maalumu lililotengwa la Visegese.Mkurugenzi Mtendaji wa halmashauri hiyo, Mussa Gama amesema ofisini kwake kuwa, eneo hilo kwenye Kata ya Kazimzumbwi lina viwanja 291 kwa ajili ya viwanda na hadi sasa ni viwanja 10 tu vimechukuliwa.Amesema, viwanda vitatu vimejengwa kwenye eneo hilo kikiwemo cha malighafi za kutengeneza sabuni, kiwanda cha kutengeneza saruji na kiwanda cha mabaki ya vifaa vya kielektroniki.Amesema hakuna urasimu katika kupata viwanja vya kujenga viwanda kwenye eneo hilo.“Unakuja katika halmashauri yetu ya wilaya ya Kisarawe ambako utakutana na maafisa ardhi wetu, watakuonyesha kwenye ramani, watakupeleka kwenye eneo la viwanda na uchagua kiwanja kinachofaa kutokana na mahitaji yako na baada ya hapo utaendelea na taratibu za umilikishwaji” amesema.Gama amesema wanatarajia Maonesho ya Pili ya Viwanda Mkoa wa Pwani na Kongamano la Fursa za Biashara na Uwekezaji vitaongeza wigo wa kupata wawekezaji kwenye sekta ya viwanda na nyinginezo.Maonesho hayo yatafanyika kwa wiki moja kuanzia Oktoba Mosi mwaka huu kwenye viwanja vya CCM Saba saba Kibaha na kongamano lifanyika Oktoba tatu.",
    "RIO DE JANEIRO, BRAZIL RAIS mteule wa Brazil mwenye siasa kali za mrengo wa kulia Jair Bolsonaro jana alikutana kwa mara ya kwanza na Waziri Mkuu wa Israel Benjamin Netanyahu mjini Rio de Janeiro na kukubaliana kuongeza ushirikiano baina ya nchi zao. Licha ya kukutana na kufanya mazungumzo  baina ya viongozi hao wawili, lakini hakuna tangazo lililotolewa kama ilivyotarajiwa na baadhi ya watu kuhusu kuhamisha ubalozi wa Brazil na kupeleka mjini Jerusalem. Ziara hiyo inamfanya Benjamin Netanyahu kuwa waziri mkuu wa kwanza wa Israel kuitembelea Brazil na kukutana na rais Bolsonaro ambaye atachukua rasmi madaraka kuanzia Januari mosi baada ya kushinda uchaguzi wa Oktoba. Bolsonaro na maafisa wake wakuu wamesema mara kadhaa kwamba watahamisha ubalozi wa Brazil kutoka mjini Tel Aviv na kupeleka Jerusalem. Lakini tangu kuchaguliwa kwake ameshinikizwa hasa na wadau katika sekta ya kilimo wanaohisi kwamba uamuzi huo utayarudisha chini mauzo yao ya nyama ambayo ni halali katika mataifa ya Kiarabu. Kulingana na barua iliyoonekana na shirika la habari la Reuters mapema Desemba, muungano wa nchi za kiarabu ulimwambia rais huyo mteule kwamba kuuhamisha ubalozi huo kutaathiri uhusiano wa nchi yake nya nchi za Kiarabu.",
    "RAIS John Magufuli jana aliungana na maaskofu, mapadri, watawa na waumini wa Kanisa Katoliki, kutoa heshima za mwisho kwa aliyekuwa Askofu wa Jimbo Katoliki la Mbeya, Askofu Evaristo Chengula.Chengula alifariki dunia Novemba 21, mwaka huu katika Taasisi ya Moyo ya Jakaya Kikwete (JKCI) Jijini Dar es Salaam, alikokuwa akipatiwa matibabu.Misa Takatifu ya kumuaga Askofu Chengula iliyoongozwa na Askofu Mkuu Mwandamizi, Jude Thadaeus Ruwa’ichi, ilifanyika katika kanisa lililopo Makao Makuu ya Baraza la Maaskofu, Kurasini Jijini Dar es Salaam na kuhudhuriwa pia na Rais mstaafu wa Awamu ya Tatu, Benjamin Mkapa na mkewe, Mama Anna.Akizungumza baada ya misa hiyo, Rais Magufuli alitoa salamu za pole kwa maaskofu wote wa Kanisa Katoliki, mapadri, watawa, waumini wa Kanisa Katoliki Jimbo la Mbeya na wakatoliki wote nchini, kwa kuondokewa na kiongozi aliyekuwa akifanya vyema kazi yake ya kutangaza injili.Rais Magufuli alisema pamoja na majukumu yake ya kutoa huduma za kiroho, Askofu Chengula alisimamia ukweli ikiwemo kupinga hadharani ndoa na mapenzi ya jinsia moja licha ya kuwepo mashinikizo mbalimbali.Akitoa salamu kwa niaba ya TEC, Makamu wa rais wa TEC, Askofu Flavian Kasalla alimshukuru Rais Magufuli kwa ushirikiano alioutoa wakati wa kuuguza na baada ya kifo cha Askofu Chengula. Pia, aliwashukuru madaktari na wauguzi, viongozi na wote waliokuwa wakimuombea kabla na baada ya kifo.",
]

# Corresponding English translations
english_translations = [
    "Kitaifa",
    "Michezo",
    "Kitaifa",
    "Kitaifa",
    "Burudani",
    "Afya",
    "Uchumi",
    "Kimtaifa",
    "Kitaifa",
]

# Process each sentence and display the highlighted text with translation
all_html_content = "<html><body>"
for idx, (sw_sentence, en_translation) in enumerate(zip(swahili_sentences, english_translations)):
    print(f"Sentence {idx+1}:")
    # Print the original Swahili sentence
    print("Original Swahili Sentence:")
    print(sw_sentence)

    # Classify the sentence (True/False)
    classification = classify_sentence(sw_sentence, model, tokenizer)
    print(f"Classification: {classification}")

    # Generate the highlighted text
    html_text = highlight_text(sw_sentence, model, tokenizer)

    # Display the highlighted text
    display(HTML(html_text))

    # Print the English translation
    print("English Translation:")
    print(en_translation)
    print("\n")
    
    # Add to the HTML content
    all_html_content += f"<h3>Sentence {idx+1}:</h3>"
    all_html_content += f"<p><strong>Original Swahili Sentence:</strong><br>{sw_sentence}</p>"
    all_html_content += f"<p><strong>Highlighted Sentence:</strong><br>{html_text}</p>"
    all_html_content += f"<p><strong>English Translation:</strong><br>{en_translation}</p>"
    all_html_content += f"<p><strong>Classification:</strong> {classification}</p>"
    all_html_content += "<hr>"

all_html_content += "</body></html>"

# Save all sentences to an HTML file
with open('attention_sentences.html', 'w', encoding='utf-8') as f:
    f.write(all_html_content)

print("All sentences have been saved to attention_sentences.html")


/tmp/ipykernel_1600144/4078459749.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Sentence 1:
Original Swahili Sentence:
BUNGE limehakikishiwa kuwa hakuna changamoto inayoweza kufarakanisha, kutenganisha wala kurudisha nyuma Muungano wa Tanganyika na Zanzibar; na kwamba badala yake unaendelea kuimarika na kushamiri.Aidha, Waziri wa Nchi, Ofisi ya Makamu wa Rais- Muungano na Mazingira, January Makamba ameasa dhidi ya wanaotumia changamoto kuupa jina baya au kuhoji uhalali wa Muungano. January alitoa msimamo huo jana wakati akiwasilisha bungeni Makadirio ya Mapato na Matumizi kwa Mwaka 2019/20 na kusema changamoto zilikuwapo, zipo na zitaendelea kuwapo isipokuwa jambo la muhimu ni kuwa na utaratibu bora wa kuzishughulikia.Alisema chini ya uongozi wa Rais John Magufuli, Makamu wa Rais, Samia Suluhu Hassan na Rais wa Zanzibar, Dk Ali Mohamed Shein, ofisi na serikali kwa ujumla itaendelea kufanyia kazi mambo yote yanayoleta changamoto katika utekelezaji wa masuala yanayohusu muungano. “Chini ya uongozi wao hakuna changamoto yoyote inayoweza kutufarakanisha, kututenganish

English Translation:
Kitaifa


Sentence 2:
Original Swahili Sentence:
Mwenyekiti wa Kamati ya Maandalizi ya tamasha hilo, Alex Msama alisema jana kuwa amezungumza na waimbaji hao na wamekubaliana watatunga wimbo maalum watakaouimba kwa pamoja siku hiyo. Msama alisema wimbo huo utaitwa Amani na Upendo na kwamba maandalizi yanaendelea kuhakikisha wasanii wengi maarufu wanakuwepo.“Watakuwa wasanii maarufu, naamini wengi waliokuwepo kwenye Tamasha la Pasaka mwaka huu watashiriki katika wimbo huo. “Tunaendelea na mazungumzo na wasanii mbalimbali, ambapo tayari Rose Muhando ameshakubali kutumbuiza na wengine tuliowatangaza,” alisema Msama.Baadhi ya wasanii maarufu waliowahi kushiriki matamasha yanayoandaliwa na Msama ni Rose Muhando, Rebecca Malope, Christina Shusho, Upendo Nkone, Upendo Kilahiro, Solomon Mukubwa, Atosha Kissava, Ephraim Sekeleti na kundi la Glorious Celebration.Pia kwaya mahiri ya Kinondoni Revival, Mwinjilisti Faraja Ntaboba kutoka Jamhuri ya Kidemokrasia ya Congo (DRC) na

English Translation:
Michezo


Sentence 3:
Original Swahili Sentence:
WAZ IR I Mkuu, Kassim Majaliwa ameondoka juzi jioni kwenda nchini China kwa ajili ya kuhudhuria mkutano wa Wakuu wa Nchi na Serikali wa Jukwaa la Ushirikiano kati ya China na Afrika (FOCAC).Mkutano huo wa siku mbili, unatarajiwa kuanza kesho Jumatatu jijini Beijing nchini China kwa kufunguliwa na Rais wa China, Xi Jinping.Katika mkutano huo, Waziri Mkuu anamwakilisha Rais John Magufuli.Kwa mujibu wa taarifa ya Ofisi ya Waziri Mkuu, mbali na kuhudhuria mkutano huo pia, Waziri Mkuu anatarajia kukutana na kufanya mazungumzo na viongozi wakuu wa kampuni mbalimbali akiwemo Rais wa Kampuni ya CCECC, Rais wa Shirika la Norinco, Rais wa Kampuni ya Zijin Gold Mine na Mwenyekiti wa Kampuni ya Saruji ya Hegnya.
Classification: True


English Translation:
Kitaifa


Sentence 4:
Original Swahili Sentence:
Na Raymond Minja Iringa Upande wa Jamhuri katika kesi ya Mwenyekiti wa Mtandao wa Wanafunzi Tanzania (TSNP), Abdul Nondo, Veronica Fredy anatarajiwa kuletwa makamani kutoa ushahidi katika kesi inayomkabili mwanafunzi huyo. Mbali na mpenzi huyo wa Nondo, Jamhuri pia inakusudia kuleta mashahidi wengine wanne ambao ni Koplo Salum, Alphonce Mwamule, Koplo Abdulkadir na mfanyakazi kutoka Kampuni ya Simu za Mkononi ya Tigo. Kesi hiyo ilikuja leo mbele ya hakimu Mkazi wa Wilaya ya Iringa, John Mpitanjia kwa ajili ya kusomewa maelezo ya awali ambapo Wakili wa Serikali, Abel Mwandalamo amesema Nondo anakabiliwa na mashtaka mawili ambayo ni kusambaza taarifa za uongo akiwa Ubungo jijini Dar Es Salaam na kutoa taarifa za uongo akiwa mjini Mafinga kwa askari Polisi kituo cha Mafinga Wilayani Mufindi. Wakili Mwandalamo amedai upande wa Jamhuri unakusudia kuambatanisha vielelezo vya ushahidi ambavyo ni taarifa za uchunguzi kuhusia

...

English Translation:
Burudani


Sentence 6:
Original Swahili Sentence:
MWANDISHI WETU-DODOMA      MAOFISA Lishe nchini wametakiwa kuelimisha wananchi kuhusu umuhimu wa kuzingatia lishe bora ili kuondoa changamoto ya utapiamlo na udumavu katika jamii. Akizungumza wakati akifunga mkutano wa siku mbili wa wadau wa lishe uliofanyika jijini Dodoma juzi, kwa niaba ya Waziri wa nchi, Ofisi Waziri mkuu, Kazi Vijana Ajira na Wenye Ulemavu, Jenista Mhagama, Katibu Mkuu Ofisi ya Waziri Mkuu anayeshughulikia Sera Uratibu na Uwekezaji,Dorothy Mwaluko alisema kuwa maofisa lishe wanapaswa kuongeza juhudi katika kutoa elimu kwa kushirikiana na wadau kuhusu umuhimu wa lishe bora. “Kipaumbele lazima kiwekwe katika makundi maalum kama wazee, watoto, akinamama wanajawazito na wale wanaonyonyesha ili kufanikisha azma ya Serikali kuondoa changamoto ya utapiamlo na udumavu hapa nchini,”alisema  Mwaluko Alisema kuwa Serikali inasisitiza matumizi ya sanaa, vyombo vya habari na viongozi wa kisiasa wakiwemo wabu

English Translation:
Afya


Sentence 7:
Original Swahili Sentence:
Halmashauri ya Wilaya ya Kisarawe inakaribisha wawekezaj wakajenge viwanda kwenye eneo maalumu lililotengwa la Visegese.Mkurugenzi Mtendaji wa halmashauri hiyo, Mussa Gama amesema ofisini kwake kuwa, eneo hilo kwenye Kata ya Kazimzumbwi lina viwanja 291 kwa ajili ya viwanda na hadi sasa ni viwanja 10 tu vimechukuliwa.Amesema, viwanda vitatu vimejengwa kwenye eneo hilo kikiwemo cha malighafi za kutengeneza sabuni, kiwanda cha kutengeneza saruji na kiwanda cha mabaki ya vifaa vya kielektroniki.Amesema hakuna urasimu katika kupata viwanja vya kujenga viwanda kwenye eneo hilo.“Unakuja katika halmashauri yetu ya wilaya ya Kisarawe ambako utakutana na maafisa ardhi wetu, watakuonyesha kwenye ramani, watakupeleka kwenye eneo la viwanda na uchagua kiwanja kinachofaa kutokana na mahitaji yako na baada ya hapo utaendelea na taratibu za umilikishwaji” amesema.Gama amesema wanatarajia Maonesho ya Pili ya Viwanda Mkoa wa Pwani na Ko

English Translation:
Uchumi


Sentence 8:
Original Swahili Sentence:
RIO DE JANEIRO, BRAZIL RAIS mteule wa Brazil mwenye siasa kali za mrengo wa kulia Jair Bolsonaro jana alikutana kwa mara ya kwanza na Waziri Mkuu wa Israel Benjamin Netanyahu mjini Rio de Janeiro na kukubaliana kuongeza ushirikiano baina ya nchi zao. Licha ya kukutana na kufanya mazungumzo  baina ya viongozi hao wawili, lakini hakuna tangazo lililotolewa kama ilivyotarajiwa na baadhi ya watu kuhusu kuhamisha ubalozi wa Brazil na kupeleka mjini Jerusalem. Ziara hiyo inamfanya Benjamin Netanyahu kuwa waziri mkuu wa kwanza wa Israel kuitembelea Brazil na kukutana na rais Bolsonaro ambaye atachukua rasmi madaraka kuanzia Januari mosi baada ya kushinda uchaguzi wa Oktoba. Bolsonaro na maafisa wake wakuu wamesema mara kadhaa kwamba watahamisha ubalozi wa Brazil kutoka mjini Tel Aviv na kupeleka Jerusalem. Lakini tangu kuchaguliwa kwake ameshinikizwa hasa na wadau katika sekta ya kilimo wanaohisi kwamba uamuzi huo utayarudis

English Translation:
Kimtaifa


Sentence 9:
Original Swahili Sentence:
RAIS John Magufuli jana aliungana na maaskofu, mapadri, watawa na waumini wa Kanisa Katoliki, kutoa heshima za mwisho kwa aliyekuwa Askofu wa Jimbo Katoliki la Mbeya, Askofu Evaristo Chengula.Chengula alifariki dunia Novemba 21, mwaka huu katika Taasisi ya Moyo ya Jakaya Kikwete (JKCI) Jijini Dar es Salaam, alikokuwa akipatiwa matibabu.Misa Takatifu ya kumuaga Askofu Chengula iliyoongozwa na Askofu Mkuu Mwandamizi, Jude Thadaeus Ruwa’ichi, ilifanyika katika kanisa lililopo Makao Makuu ya Baraza la Maaskofu, Kurasini Jijini Dar es Salaam na kuhudhuriwa pia na Rais mstaafu wa Awamu ya Tatu, Benjamin Mkapa na mkewe, Mama Anna.Akizungumza baada ya misa hiyo, Rais Magufuli alitoa salamu za pole kwa maaskofu wote wa Kanisa Katoliki, mapadri, watawa, waumini wa Kanisa Katoliki Jimbo la Mbeya na wakatoliki wote nchini, kwa kuondokewa na kiongozi aliyekuwa akifanya vyema kazi yake ya kutangaza injili.Rais Magufuli alisema pa

English Translation:
Kitaifa


All sentences have been saved to attention_sentences.html
